In [1]:
import time
from ccd.procedures import fit_procedure as __determine_fit_procedure
import numpy as np
import pandas as pd
from ccd import app, math_utils, qa, __check_inputs, __sort_dates, attr_from_str
from datetime import datetime
import importlib

from test.shared import rainbow

In [2]:
requested_ubids = ['LANDSAT_4/TM/SRB1', 'LANDSAT_4/TM/SRB2', 'LANDSAT_4/TM/SRB3', 'LANDSAT_4/TM/SRB4', 'LANDSAT_4/TM/SRB5', 'LANDSAT_4/TM/BTB6', 'LANDSAT_4/TM/SRB7', 'LANDSAT_4/TM/PIXELQA', 'LANDSAT_5/TM/SRB1', 'LANDSAT_5/TM/SRB2', 'LANDSAT_5/TM/SRB3', 'LANDSAT_5/TM/SRB4', 'LANDSAT_5/TM/SRB5', 'LANDSAT_5/TM/BTB6', 'LANDSAT_5/TM/SRB7', 'LANDSAT_5/TM/PIXELQA', 'LANDSAT_7/ETM/SRB1', 'LANDSAT_7/ETM/SRB2', 'LANDSAT_7/ETM/SRB3', 'LANDSAT_7/ETM/SRB4', 'LANDSAT_7/ETM/SRB5', 'LANDSAT_7/ETM/BTB6', 'LANDSAT_7/ETM/SRB7', 'LANDSAT_7/ETM/PIXELQA', 'LANDSAT_8/OLI_TIRS/SRB2', 'LANDSAT_8/OLI_TIRS/SRB3', 'LANDSAT_8/OLI_TIRS/SRB4', 'LANDSAT_8/OLI_TIRS/SRB5', 'LANDSAT_8/OLI_TIRS/SRB6', 'LANDSAT_8/OLI_TIRS/SRB7', 'LANDSAT_8/OLI_TIRS/BTB10', 'LANDSAT_8/OLI_TIRS/PIXELQA']
specs_url  = 'foo'
chips_url  = 'bar'
acquired   = '1982-01-01/2015-12-31'
x, y       = -2094585, 1682805
xind, yind = 57, 97
row, col   = yind, xind

In [3]:
rbow = rainbow(x, y, acquired, specs_url, chips_url, requested_ubids)

In [4]:
proc_params = app.get_default_params()

In [5]:
# dates
def dtstr_to_ordinal(dtstr, iso=True):
    """ Return ordinal from string formatted date"""
    _fmt = '%Y-%m-%dT%H:%M:%SZ' if iso else '%Y-%m-%d %H:%M:%S'
    _dt = datetime.strptime(dtstr, _fmt)
    return _dt.toordinal()


rainbow_date_array = np.array(rbow['t'].values)
_dates = [dtstr_to_ordinal(str(pd.to_datetime(i)), False) for i in rainbow_date_array]

# date handling with ccd.__init__::detect
dates = np.asarray(_dates)

In [6]:
# quality
_quality = np.array(rbow['cfmask'].values[:, row, col], dtype=int)

# quality handling with ccd.__init__::detect
quality = np.asarray(_quality)

In [7]:
# how bands passed to ccd.detect()
blue    = np.array(rbow['blue'].values[:, row, col])
green   = np.array(rbow['green'].values[:, row, col])
red     = np.array(rbow['red'].values[:, row, col])
nir     = np.array(rbow['nir'].values[:, row, col])
swir1   = np.array(rbow['swir1'].values[:, row, col])
swir2   = np.array(rbow['swir2'].values[:, row, col])
thermal = np.array(rbow['thermal'].values[:, row, col])

# how bands org'd within ccd.detect
spectra = np.stack((blue, green, red, nir, swir1, swir2, thermal))

In [8]:
spectra

array([[ 20000.,      0.,    483., ...,      0.,      0.,      0.],
       [  7767.,      0.,    568., ...,      0.,      0.,      0.],
       [  7994.,      0.,    553., ...,      0.,      0.,      0.],
       ..., 
       [  5586.,      0.,    151., ...,      0.,      0.,      0.],
       [  4266.,      0.,    220., ...,      0.,      0.,      0.],
       [  2477.,      0.,   2822., ...,      0.,      0.,      0.]])

In [9]:
__check_inputs(dates, quality, spectra)

In [10]:
indices = __sort_dates(dates)
dates = dates[indices]
spectra = spectra[:, indices]
quality = quality[indices]

In [11]:
if proc_params.QA_BITPACKED is True:
    quality = qa.unpackqa(quality, proc_params)

In [12]:
procedure = __determine_fit_procedure(quality, proc_params)

In [13]:
procedure

CPUDispatcher(<function standard_procedure at 0x7efdec374c80>)

In [14]:
fitter_fn = attr_from_str(proc_params.FITTER_FN)

In [15]:
proc_params.FITTER_FN

'ccd.models.lasso.fitted_model'

In [16]:
#results = procedure(dates, spectra, fitter_fn, quality, proc_params)

In [17]:
from ccd import qa
from ccd.change import enough_samples, enough_time,\
    update_processing_mask, stable, determine_num_coefs, calc_residuals, \
    find_closest_doy, change_magnitude, detect_change, detect_outlier
from ccd.models import results_to_changemodel, tmask
from ccd.math_utils import kelvin_to_celsius, adjusted_variogram, euclidean_norm

from numba import jit

In [78]:
from collections import namedtuple

ChangeModel = namedtuple('ChangeModel', ['start_day',
                                         'end_day',
                                         'break_day',
                                         'observation_count',
                                         'change_probability',
                                         'curve_qa',
                                         'blue',
                                         'green',
                                         'red',
                                         'nir',
                                         'swir1',
                                         'swir2',
                                         'thermal'])


#@jit
def lookforward2(dates, observations, model_window, fitter_fn, processing_mask, variogram, proc_params):
    peek_size       = proc_params.PEEK_SIZE
    coef_min        = proc_params.COEFFICIENT_MIN
    coef_mid        = proc_params.COEFFICIENT_MID
    coef_max        = proc_params.COEFFICIENT_MAX
    num_obs_fact    = proc_params.NUM_OBS_FACTOR
    detection_bands = proc_params.DETECTION_BANDS
    change_thresh   = proc_params.CHANGE_THRESHOLD
    outlier_thresh  = proc_params.OUTLIER_THRESHOLD
    avg_days_yr     = proc_params.AVG_DAYS_YR
    fit_max_iter    = proc_params.LASSO_MAX_ITER

    # Step 4: lookforward.
    # The second step is to update a model until observations that do not
    # fit the model are found.
    #log.debug('lookforward initial model window: %s', model_window)

    # The fit_window pertains to which locations are used in the model
    # regression, while the model_window identifies the locations in which
    # fitted models apply to. They are not always the same.
    fit_window = model_window

    # Initialized for a check at the first iteration.
    models = None

    # Simple value to determine if change has occured or not. Change may not
    # have occurred if we reach the end of the time series.
    change = 0

    # Initial subset of the data
    period       = dates[processing_mask]
    spectral_obs = observations[:, processing_mask]

    # Used for comparison purposes
    fit_span = period[model_window.stop - 1] - period[model_window.start]

    # stop is always exclusive
    while model_window.stop + peek_size < period.shape[0] or models is None:
        
        num_coefs = determine_num_coefs(period[model_window], coef_min,
                                        coef_mid, coef_max, num_obs_fact)
        #print("num_coefs type, val: {} {}".format(type(num_coefs), num_coefs))
        #num_coefs = 4
        
        peek_window = slice(model_window.stop, model_window.stop + peek_size)

        # Used for comparison against fit_span
        model_span = period[model_window.stop - 1] - period[model_window.start]
        
        #-----
        
        if not models or model_window.stop - model_window.start < 24:
            fit_span = period[model_window.stop - 1] - period[model_window.start]

            fit_window = model_window
            #log.debug('Retrain models, less than 24 samples')
            #models = [fitter_fn(period[fit_window], spectrum,
            #                    fit_max_iter, avg_days_yr, num_coefs)
            #          for spectrum in spectral_obs[:, fit_window]]
            models = []
            for spectrum in spectral_obs[:, fit_window]:
                models.append(fitter_fn(period[fit_window], spectrum, fit_max_iter, avg_days_yr, num_coefs))

            # residuals = np.array([calc_residuals(period[peek_window],
            #                                      spectral_obs[idx, peek_window],
            #                                      models[idx], avg_days_yr)
            #                       for idx in range(observations.shape[0])])
            _residuals = []
            for idx in range(observations.shape[0]):
                _residuals.append(calc_residuals(period[peek_window],
                                                 spectral_obs[idx, peek_window],
                                                 models[idx], avg_days_yr))
            residuals = np.array(_residuals)

            #comp_rmse = [models[idx].rmse for idx in detection_bands]
            comp_rmse = []
            for idx in detection_bands:
                comp_rmse.append(models[idx].rmse)


        # More than 24 points
        else:
            # If the number of observations that the current fitted models
            # expand past a threshold, then we need to fit new ones.
            # The 1.33 should be parametrized at some point.
            if model_span >= 1.33 * fit_span:
                #log.debug('Retrain models, model_span: %s fit_span: %s', model_span, fit_span)
                fit_span = period[model_window.stop - 1] - period[model_window.start]
                fit_window = model_window

                #models = [fitter_fn(period[fit_window], spectrum,
                #                    fit_max_iter, avg_days_yr, num_coefs)
                #          for spectrum in spectral_obs[:, fit_window]]
                models = []
                for spectrum in spectral_obs[:, fit_window]:
                    models.append(fitter_fn(period[fit_window],
                                            spectrum,
                                            fit_max_iter,
                                            avg_days_yr,
                                            num_coefs))
                    
             # residuals = np.array([calc_residuals(period[peek_window],
            #                                      spectral_obs[idx, peek_window],
            #                                      models[idx], avg_days_yr)
            #                       for idx in range(observations.shape[0])])
            _residuals = []
            for idx in range(observations.shape[0]):
                _residuals.append(calc_residuals(period[peek_window],
                                                 spectral_obs[idx, peek_window],
                                                 models[idx], avg_days_yr))
            residuals = np.array(_residuals)
            
            # We want to use the closest residual values to the peek_window
            # values based on seasonality.
            closest_indexes = find_closest_doy(period, peek_window.stop - 1, fit_window, 24)

            # Calculate an RMSE for the seasonal residual values, using 8
            # as the degrees of freedom.
            # comp_rmse = [euclidean_norm(models[idx].residual[closest_indexes]) / 4
            #              for idx in detection_bands]
            comp_rmse = []
            for idx in detection_bands:
                comp_rmse.append(euclidean_norm(models[idx].residual[closest_indexes]) / 4)
                
                
        # Calculate the change magnitude values for each observation in the
        # peek_window.
        magnitude = change_magnitude(residuals[detection_bands, :],
                                     variogram[detection_bands],
                                     comp_rmse)
        
        if detect_change(magnitude, change_thresh):
            #log.debug('Change detected at: %s', peek_window.start)

            # Change was detected, return to parent method
            change = 1
            break
        elif detect_outlier(magnitude[0], outlier_thresh):
            #log.debug('Outlier detected at: %s', peek_window.start)

            # Keep track of any outliers so they will be excluded from future
            # processing steps
            processing_mask = update_processing_mask(processing_mask,
                                                     peek_window.start)

            # Because only one value was excluded, we shouldn't need to adjust
            # the model_window.  The location hasn't been used in
            # processing yet. So, the next iteration can use the same windows
            # without issue.
            period = dates[processing_mask]
            spectral_obs = observations[:, processing_mask]
            continue

        model_window = slice(model_window.start, model_window.stop + 1)
    
    print("** residuals: {}".format(residuals))
    nc = determine_num_coefs(period[model_window], coef_min, coef_mid, coef_max, num_obs_fact)
    result = results_to_changemodel(fitted_models=models,
                                    start_day=period[model_window.start],
                                    end_day=period[model_window.stop - 1],
                                    break_day=period[model_window.stop],
                                    magnitudes=np.median(residuals),
                                    observation_count=(model_window.stop - model_window.start),
                                    change_probability=change,
                                    curve_qa=nc)

        
        #------
    #s = peek_window.start    
    #result = ChangeModel(start_day=period[model_window.start],
    #                     end_day=period[model_window.stop - 1],
    #                     break_day=model_window.stop, # jit balks at peek_window.start 
    #                     observation_count=(model_window.stop - model_window.start),
    #                     change_probability=change,
    #                     curve_qa=determine_num_coefs(period[model_window], coef_min, coef_mid, coef_max, num_obs_fact),
    #                     blue=6,
    #                     green=6,
    #                     red=6,
    #                     nir=6,
    #                     swir1=6,
    #                     swir2=6,
    #                     thermal=6)    
        
        
                
    return processing_mask, model_window, result
    

In [20]:
observations = spectra
model_window = slice(24, 36, None)

In [21]:
#mock a processing_mask
processing_mask = []
for i in range(0, 2809):
    if i % 2:
        processing_mask.append(True)
    else:
        processing_mask.append(False)
processing_mask = np.array(processing_mask)

In [22]:
variogram = np.array([ 161,  227,   304,  331,  399,  328,  820], dtype=float)

In [93]:
@jit
def lookforward(dates, observations, model_window, fitter_fn, processing_mask,
                variogram, proc_params):
    """Increase observation window until change is detected or
    we are out of observations.

    Args:
        dates: list of ordinal day numbers relative to some epoch,
            the particular epoch does not matter.
        observations: spectral values, list of spectra -> values
        model_window: span of indices that is represented in the current
            process
        fitter_fn: function used to model observations
        processing_mask: 1-d boolean array identifying which values to
            consider for processing
        variogram: 1-d array of variogram values to compare against for the
            normalization factor
        proc_params: dictionary of processing parameters

    Returns:
        namedtuple: representation of the time segment
        1-d bool ndarray: processing mask that may have been modified
        slice: model window
    """
    # TODO do this better
    peek_size       = proc_params.PEEK_SIZE
    coef_min        = proc_params.COEFFICIENT_MIN
    coef_mid        = proc_params.COEFFICIENT_MID
    coef_max        = proc_params.COEFFICIENT_MAX
    num_obs_fact    = proc_params.NUM_OBS_FACTOR
    detection_bands = proc_params.DETECTION_BANDS
    change_thresh   = proc_params.CHANGE_THRESHOLD
    outlier_thresh  = proc_params.OUTLIER_THRESHOLD
    avg_days_yr     = proc_params.AVG_DAYS_YR
    fit_max_iter    = proc_params.LASSO_MAX_ITER
    
    peek_window = None
    residuals = None
    num_coefs = None

    # Step 4: lookforward.
    # The second step is to update a model until observations that do not
    # fit the model are found.
    #log.debug('lookforward initial model window: %s', model_window)

    # The fit_window pertains to which locations are used in the model
    # regression, while the model_window identifies the locations in which
    # fitted models apply to. They are not always the same.
    fit_window = model_window

    # Initialized for a check at the first iteration.
    models = None

    # Simple value to determine if change has occured or not. Change may not
    # have occurred if we reach the end of the time series.
    change = 0

    # Initial subset of the data
    period       = dates[processing_mask]
    spectral_obs = observations[:, processing_mask]

    # Used for comparison purposes
    fit_span = period[model_window.stop - 1] - period[model_window.start]

    # stop is always exclusive
    while model_window.stop + peek_size < period.shape[0] or models is None:
    #while model_window.stop + peek_size < period.shape[0]:
        print("model_window.stop: {}".format(model_window.stop))
        print("model_window type: {}".format(type(model_window)))
        print("peek_size: {}".format(peek_size))
        print("period_shape[0]: {}".format(period.shape[0]))
        print("period_shape type: {}".format(type(period)))
        print("models is None: {}".format(models is None))
        num_coefs = determine_num_coefs(period[model_window], coef_min,
                                        coef_mid, coef_max, num_obs_fact)
        #print("num_coefs type, val: {} {}".format(type(num_coefs), num_coefs))
        #num_coefs = 4
        
        peek_window = slice(model_window.stop, model_window.stop + peek_size)

        # Used for comparison against fit_span
        model_span = period[model_window.stop - 1] - period[model_window.start]

        #log.debug('Detecting change for %s', peek_window)

        # If we have less than 24 observations covered by the model_window
        # or it the first iteration, then we always fit a new window
        if not models or model_window.stop - model_window.start < 24:
            fit_span = period[model_window.stop - 1] - period[model_window.start]

            fit_window = model_window
            #log.debug('Retrain models, less than 24 samples')
            #models = [fitter_fn(period[fit_window], spectrum,
            #                    fit_max_iter, avg_days_yr, num_coefs)
            #          for spectrum in spectral_obs[:, fit_window]]
            models = []
            for spectrum in spectral_obs[:, fit_window]:
                models.append(fitter_fn(period[fit_window], spectrum, fit_max_iter, avg_days_yr, num_coefs))

            # residuals = np.array([calc_residuals(period[peek_window],
            #                                      spectral_obs[idx, peek_window],
            #                                      models[idx], avg_days_yr)
            #                       for idx in range(observations.shape[0])])
            _residuals = []
            for idx in range(observations.shape[0]):
                _residuals.append(calc_residuals(period[peek_window],
                                                 spectral_obs[idx, peek_window],
                                                 models[idx], avg_days_yr))
            residuals = np.array(_residuals)

            #comp_rmse = [models[idx].rmse for idx in detection_bands]
            comp_rmse = []
            for idx in detection_bands:
                comp_rmse.append(models[idx].rmse)


        # More than 24 points
        else:
            # If the number of observations that the current fitted models
            # expand past a threshold, then we need to fit new ones.
            # The 1.33 should be parametrized at some point.
            if model_span >= 1.33 * fit_span:
                #log.debug('Retrain models, model_span: %s fit_span: %s', model_span, fit_span)
                fit_span = period[model_window.stop - 1] - period[model_window.start]
                fit_window = model_window

                #models = [fitter_fn(period[fit_window], spectrum,
                #                    fit_max_iter, avg_days_yr, num_coefs)
                #          for spectrum in spectral_obs[:, fit_window]]
                models = []
                for spectrum in spectral_obs[:, fit_window]:
                    models.append(fitter_fn(period[fit_window],
                                            spectrum,
                                            fit_max_iter,
                                            avg_days_yr,
                                            num_coefs))


            # residuals = np.array([calc_residuals(period[peek_window],
            #                                      spectral_obs[idx, peek_window],
            #                                      models[idx], avg_days_yr)
            #                       for idx in range(observations.shape[0])])
            _residuals = []
            for idx in range(observations.shape[0]):
                _residuals.append(calc_residuals(period[peek_window],
                                                 spectral_obs[idx, peek_window],
                                                 models[idx], avg_days_yr))
            residuals = np.array(_residuals)


            # We want to use the closest residual values to the peek_window
            # values based on seasonality.
            closest_indexes = find_closest_doy(period, peek_window.stop - 1, fit_window, 24)

            # Calculate an RMSE for the seasonal residual values, using 8
            # as the degrees of freedom.
            # comp_rmse = [euclidean_norm(models[idx].residual[closest_indexes]) / 4
            #              for idx in detection_bands]
            comp_rmse = []
            for idx in detection_bands:
                comp_rmse.append(euclidean_norm(models[idx].residual[closest_indexes]) / 4)

        # Calculate the change magnitude values for each observation in the
        # peek_window.
        magnitude = change_magnitude(residuals[detection_bands, :],
                                     variogram[detection_bands],
                                     comp_rmse)

        if detect_change(magnitude, change_thresh):
            #log.debug('Change detected at: %s', peek_window.start)

            # Change was detected, return to parent method
            change = 1
            break
        elif detect_outlier(magnitude[0], outlier_thresh):
            #log.debug('Outlier detected at: %s', peek_window.start)

            # Keep track of any outliers so they will be excluded from future
            # processing steps
            processing_mask = update_processing_mask(processing_mask,
                                                     peek_window.start)

            # Because only one value was excluded, we shouldn't need to adjust
            # the model_window.  The location hasn't been used in
            # processing yet. So, the next iteration can use the same windows
            # without issue.
            period = dates[processing_mask]
            spectral_obs = observations[:, processing_mask]
            continue

        model_window = slice(model_window.start, model_window.stop + 1)
    #num_coefs = determine_num_coefs(period[model_window], coef_min, coef_mid, coef_max, num_obs_fact)
    result = results_to_changemodel(fitted_models=models,
                                    start_day=period[model_window.start],
                                    end_day=period[model_window.stop - 1],
                                    break_day=period[peek_window.start],
                                    magnitudes=np.median(residuals),
                                    observation_count=(model_window.stop - model_window.start),
                                    change_probability=change,
                                    curve_qa=num_coefs)

    return result, processing_mask, model_window

In [94]:
lf = lookforward(dates, observations, model_window, fitter_fn, processing_mask, variogram, proc_params)

model_window.stop: 36
model_window type: <class 'slice'>
peek_size: 6
period_shape[0]: 1403
period_shape type: <class 'numpy.ndarray'>
models is None: True
model_window.stop: 37
model_window type: <class 'slice'>
peek_size: 6
period_shape[0]: 1403
period_shape type: <class 'numpy.ndarray'>
models is None: False


IndexError: invalid index to scalar variable.

In [67]:
dates

array([723864, 723873, 723880, ..., 735959, 735960, 735961])